Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

* Наименование вакансии.
* Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import math
import re
import warnings

warnings.filterwarnings("ignore")

In [2]:
# headers = {'User-Agent': 
#            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 YaBrowser/19.10.3.281 Yowser/2.5 Safari/537.36'}
data_list = []

In [3]:
def hh_search():
    
    def add_column():
        vacancy_data['link'] = vacancy_link
        vacancy_data['name'] = vacancy_name
        vacancy_data['min'] = min_sallary
        vacancy_data['max'] = max_sallary
        vacancy_data['currency'] = currency
        vacancy_data['site'] = site
        data_list.append(vacancy_data)
    
    base_hh_url = 'https://hh.ru'
    for i in range(pages):
        full_hh_url = base_hh_url + f'/search/vacancy?area=1&fromSearchLine=true&text={search_query}&from=suggest_post&page={i}&hhtmFrom=vacancy_search_list'
        response = requests.get(full_hh_url, headers = headers)
        with open('response.html', 'w', encoding = 'utf-8') as f:
            f.write(response.text)
        html = ''
        with open('response.html', 'r', encoding = 'utf-8') as f:
            html = f.read()
        dom = bs(html, 'html.parser')
        vacancies = dom.find_all('div', {'class': "vacancy-serp-item-body"})

        for vacancy in vacancies:
            vacancy_data = {}
            vacancy_link = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title' })['href']
            vacancy_name = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title' }).getText()
            sallary = vacancy.find('span', {'data-qa' : 'vacancy-serp__vacancy-compensation'})
            if sallary:
                text = sallary.getText()

                if 'от' in text:

                    min_sallary = re.findall('[0-9]+', text)
                    min_sallary = int(''.join(min_sallary))
                    max_sallary = None
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[1]
                    site = base_hh_url
                    add_column()
                    continue

                if 'до' in text:

                    min_sallary = None
                    max_sallary = re.findall('[0-9]+', text)
                    max_sallary = int(''.join(max_sallary))
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[1]
                    site = base_hh_url
                    add_column()
                    continue

                else:

                    sallary_ = re.findall('[0-9]+', text)
                    min_sallary = int(sallary_[0] + sallary_[1])
                    max_sallary = int(sallary_[2] + sallary_[3])
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[0]
                    site = base_hh_url
                    add_column()
                    continue
            else:
                min_sallary = None
                max_sallary = None
                currency = None
                site = base_hh_url
                add_column()
                
    ###### *************************************************************************************************** #####
    
def sj_search():
    
    def add_column():
        vacancy_data['link'] = vacancy_link
        vacancy_data['name'] = vacancy_name
        vacancy_data['min'] = min_sallary
        vacancy_data['max'] = max_sallary
        vacancy_data['currency'] = currency
        vacancy_data['site'] = site
        data_list.append(vacancy_data)
        
    base_sj_url = 'https://russia.superjob.ru'
    for i in range(pages):
        full_sj_url = base_sj_url + f'/vacancy/search/?keywords={search_query}&page={i}'
        response = requests.get(full_sj_url, headers = headers, verify=False)
        with open('response.html', 'w', encoding = 'utf-8') as f:
            f.write(response.text)
        html = ''
        with open('response.html', 'r', encoding = 'utf-8') as f:
            html = f.read()
        dom = bs(html, 'html.parser')
        vacancies = dom.find_all('div', {'class': '_3JPvf jNMYr PEEVg WuWPa'})

        for vacancy in vacancies:
            vacancy_data = {}
            vacancy_link = base_sj_url + vacancy.find('a', {'class': 'icMQ_' })['href']
            #print(vacancy)
            vacancy_name = vacancy.find_all('span', {'class': '_1rS-s' })
            str_ = ''
            for name in vacancy_name:
                str_ += (' ' + name.getText())
            vacancy_name = str_
            sallary = vacancy.find('span', {'class' : '_2Wp8I _1BiPY _26ig7 OCAdW _18w_0'})
            if sallary:
                text = sallary.getText()

                if 'По договорённости' in text:
                    min_sallary = None
                    max_sallary = None
                    currency = None
                    site = base_sj_url
                    add_column()
                    continue

                if 'от' in text:

                    min_sallary = re.findall('[0-9]+', text)
                    min_sallary = int(''.join(min_sallary))
                    max_sallary = None
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[1]
                    site = base_sj_url
                    add_column()
                    continue

                if 'до' in text:

                    min_sallary = None
                    max_sallary = re.findall('[0-9]+', text)
                    max_sallary = ''.join(max_sallary)
                    cur_sallary = re.findall('[A-zА-я]+', text)
                    currency = cur_sallary[1]
                    site = base_sj_url
                    add_column()
                    continue

                else:

                    sallary_ = re.findall('[0-9]+', text)
                    if len(sallary_) > 2:
                        min_sallary = int(sallary_[0] + sallary_[1])
                        max_sallary = int(sallary_[2] + sallary_[3])
                        cur_sallary = re.findall('[A-zА-я]+', text)
                        currency = cur_sallary[0]
                        site = base_sj_url
                        add_column()
                        continue
                    else:
                        min_sallary = int(sallary_[0] + sallary_[1])
                        max_sallary = int(sallary_[0] + sallary_[1])
                        cur_sallary = re.findall('[A-zА-я]+', text)
                        currency = cur_sallary[0]
                        site = base_sj_url
                        add_column()
                        continue 
                        
    ###### *************************************************************************************************** #####
    
def get_result():
    df = pd.DataFrame()
    hh_search()
    sj_search()    

In [4]:
search_query = input('Введите название вакансии для поиска: ').replace(' ', '+')
pages = int(input('Введите количество страниц сайта, по которым необходимо собрать информацию: '))

Введите название вакансии для поиска: сантехник
Введите количество страниц сайта, по которым необходимо собрать информацию: 3


In [5]:
get_result()
df = pd.DataFrame(data = data_list)
df

,link,name,min,max,currency,site
0,https://hh.ru/vacancy/53935376?query=%D1%81%D0...,Мастер - универсал,60000.0,120000,руб,https://hh.ru
1,https://hh.ru/vacancy/54396715?query=%D1%81%D0...,Техник/Хаус-мастер,60000.0,None,руб,https://hh.ru
2,https://hh.ru/vacancy/53949944?query=%D1%81%D0...,Сантехник-монтажник,80000.0,100000,руб,https://hh.ru
3,https://hh.ru/vacancy/54524582?query=%D1%81%D0...,Техник по обслуживанию апартаментов,50000.0,60000,руб,https://hh.ru
4,https://hh.ru/vacancy/54453729?query=%D1%81%D0...,Техник по эксплуатации зданий,60000.0,None,руб,https://hh.ru
...,...,...,...,...,...,...
215,https://russia.superjob.ru/vakansii/slesar-san...,сантехник,91000.0,None,руб,https://russia.superjob.ru
216,https://russia.superjob.ru/vakansii/slesar-san...,сантехник,NaN,None,None,https://russia.superjob.ru
217,https://russia.superjob.ru/vakansii/slesar-san...,сантехник,25000.0,35000,руб,https://russia.superjob.ru
218,https://russia.superjob.ru/vakansii/slesar-san...,сантехник,50000.0,None,руб,https://russia.superjob.ru
